In [67]:
import pandas as pd
pd.set_option('display.max_colwidth',100)

In [68]:
Data=pd.read_excel('drive/My Drive/Ankit/october2020 codalab/english/Constraint_English_Train.xlsx')
Data.dropna(inplace = True)
Data.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In general the discrepancies in death counts between dif...,real
1,2,States reported 1121 deaths a small rise from last Tuesday. Southern states reported 640 of thos...,real
2,3,Politically Correct Woman (Almost) Uses Pandemic as Excuse Not to Reuse Plastic Bag https://t.co...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testing laboratories in India and as on 25th August 2020...,real
4,5,Populous states can generate large case counts but if you look at the new cases per million toda...,real


In [69]:
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
def remove_sw(txt):
    no_sw =" ".join([a for a in txt.split() if a not in stop])
    return no_sw
Data['tweet'] = Data['tweet'].apply(lambda x: remove_sw(x))
Data[['tweet','label']].head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,tweet,label
0,The CDC currently reports 99031 deaths. In general discrepancies death counts different sources ...,real
1,States reported 1121 deaths small rise last Tuesday. Southern states reported 640 deaths. https:...,real
2,Politically Correct Woman (Almost) Uses Pandemic Excuse Not Reuse Plastic Bag https://t.co/thF8G...,fake
3,#IndiaFightsCorona: We 1524 #COVID testing laboratories India 25th August 2020 36827520 tests do...,real
4,Populous states generate large case counts look new cases per million today 9 smaller states sho...,real


In [70]:
#rem0ving emoji

import re

def deEmojify(txt):
    regrex_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',txt)

Data['tweet']=Data['tweet'].apply(lambda x:deEmojify(x))
Data[['tweet','label']].head()

,tweet,label
0,The CDC currently reports 99031 deaths. In general discrepancies death counts different sources ...,real
1,States reported 1121 deaths small rise last Tuesday. Southern states reported 640 deaths. https:...,real
2,Politically Correct Woman (Almost) Uses Pandemic Excuse Not Reuse Plastic Bag https://t.co/thF8G...,fake
3,#IndiaFightsCorona: We 1524 #COVID testing laboratories India 25th August 2020 36827520 tests do...,real
4,Populous states generate large case counts look new cases per million today 9 smaller states sho...,real


In [71]:
# remove URL
def remove_URL(txt):
    url= re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"",txt)

Data['tweet']=Data['tweet'].apply(lambda x:remove_URL(x))


Data['tweet']=Data['tweet'].str.replace(r'\d+','')
Data.head()

,id,tweet,label
0,1,The CDC currently reports deaths. In general discrepancies death counts different sources small...,real
1,2,States reported deaths small rise last Tuesday. Southern states reported deaths.,real
2,3,Politically Correct Woman (Almost) Uses Pandemic Excuse Not Reuse Plastic Bag #coronavirus #nas...,fake
3,4,#IndiaFightsCorona: We #COVID testing laboratories India th August tests done : @ProfBhargava...,real
4,5,Populous states generate large case counts look new cases per million today smaller states show...,real


In [72]:
import keras
from keras.preprocessing.text import Tokenizer
tok1 = Tokenizer(char_level=False, filters ='!"$%&@()*+,-./:;”“<=>?[\\]^_`{|}~\t\n', lower = True)

In [73]:
tok1.fit_on_texts(Data['tweet'])
words=len(tok1.word_counts)
words

13718

In [74]:
word_list=[tok1.word_index]
#word_list

In [75]:
encoded=tok1.texts_to_sequences(Data['tweet'])
#print(Data['text'][0])
#encoded[1211]

In [76]:
import numpy as np
from keras.utils.np_utils import to_categorical
classes_list = ["real","fake"]
label_index = Data['label'].apply(classes_list.index)
label1 = np.asarray(label_index)
label = to_categorical(np.asarray(label1))
label

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [77]:
##         Padding encoded sequence of words
from keras.preprocessing import sequence
max_length=20
padd = sequence.pad_sequences(encoded, maxlen=max_length, padding='post')
padd[0:12]

array([[  91,  223,  209,   10,   47,  447, 6628,   88, 1166,  389, 1645,
         650, 6629,    6,   88,  615,  513, 1646,    7,   22],
       [  11,   17,   10,  650,  381,   51, 1002, 2376,   11,   17,   10,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [4721, 1776,  390,  494, 2671,   34, 2377,  301, 6630, 2378, 3135,
          19, 1777,    0,    0,    0,    0,    0,    0,    0],
       [  27,   24,    3,   18,  248,    9,  130,  252,    8,  448, 2160,
        1647,  159,  108,  136,    0,    0,    0,    0,    0],
       [ 104,   65,   22, 1957,   11,  526,    2,  104,   65,  334,  391,
        2672, 4724, 3718, 3719, 3720, 1233, 2673, 2674, 3721],
       [ 162, 1535,    1, 1536,    7,   39,   66,  172, 1537,   57, 1426,
          89, 1538,  163,  267,   77,  580,  217,  469,  533],
       [ 100,   43,   42,    3,  132,  163,   77,  436,    7,  121, 1167,
        1320,  180,  192,    1,  172,    0,    0,    0,    0],
       [ 675, 1539, 1648,    4,  280,   1

In [78]:
# importing libraries for creating model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Embedding,MaxPool1D
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D,LSTM

# Reading embedding vector using Glove 


In [79]:
embeddings_index = {}
f = open('drive/My Drive/tweeter task/Copy of glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [80]:
embedding_matrix = np.zeros((words+1, 300))
for word, i in tok1.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [81]:
Embedding_Layer = Embedding(words+1,300,weights=[embedding_matrix], input_length=max_length, trainable=False)

#lstm

In [82]:
model = Sequential()
model.add(Embedding_Layer)
model.add(LSTM(256,return_sequences=False))
model.add(Flatten())
model.add(Dense(32, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 300)           4115700   
_________________________________________________________________
lstm_5 (LSTM)                (None, 256)               570368    
_________________________________________________________________
flatten_5 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 32)                8224      
_________________________________________________________________
dense_12 (Dense)             (None, 64)                2112      
_________________________________________________________________
dense_13 (Dense)             (None, 2)                 130       
Total params: 4,696,534
Trainable params: 580,834
Non-trainable params: 4,115,700
______________________________________

In [83]:
## compile

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [84]:
model.fit(padd,label,epochs=50,verbose=1,batch_size=32)
#model.fit(x_train,y_train,epochs=50,verbose=1,batch_size=16)

Epoch 1/50
201/201 [==============================] - 18s 88ms/step - loss: 0.3451 - acc: 0.8540
Epoch 2/50
201/201 [==============================] - 17s 87ms/step - loss: 0.2442 - acc: 0.8986
Epoch 3/50
201/201 [==============================] - 18s 87ms/step - loss: 0.2038 - acc: 0.9182
Epoch 4/50
201/201 [==============================] - 18s 88ms/step - loss: 0.1488 - acc: 0.9416
Epoch 5/50
201/201 [==============================] - 18s 87ms/step - loss: 0.1044 - acc: 0.9628
Epoch 6/50
201/201 [==============================] - 18s 88ms/step - loss: 0.0785 - acc: 0.9720
Epoch 7/50
201/201 [==============================] - 17s 87ms/step - loss: 0.0596 - acc: 0.9791
Epoch 8/50
201/201 [==============================] - 17s 86ms/step - loss: 0.0490 - acc: 0.9849
Epoch 9/50
201/201 [==============================] - 17s 87ms/step - loss: 0.0241 - acc: 0.9919
Epoch 10/50
201/201 [==============================] - 18s 90ms/step - loss: 0.0281 - acc: 0.9905
Epoch 11/50
201/201 [========

#testing

In [85]:
Test =pd.read_excel('drive/My Drive/Ankit/october2020 codalab/english/Constraint_English_Val.xlsx')
Test.dropna(inplace = True)
Test.head()

,id,tweet,label
0,1,Chinese converting to Islam after realising that no muslim was affected by #Coronavirus #COVD19 ...,fake
1,2,11 out of 13 people (from the Diamond Princess Cruise ship) who had intially tested negative in ...,fake
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus And Can Be Treated With Aspirin",fake
3,4,Mike Pence in RNC speech praises Donald Trump’s COVID-19 “seamless” partnership with governors a...,fake
4,5,6/10 Sky's @EdConwaySky explains the latest #COVID19 data and government announcement. Get more ...,real


In [86]:
## doing preprocessing as above

Test['tweet'] = Test['tweet'].apply(lambda x: remove_sw(x))
Test['tweet']=Test['tweet'].apply(lambda x:deEmojify(x))
Test['tweet']=Test['tweet'].apply(lambda x: remove_URL(x))
Test['tweet']=Test['tweet'].str.replace(r'\d+','')
t= Test['tweet']

In [87]:
encoded1 =tok1.texts_to_sequences(t)
print(t[0])
encoded1[0]

Chinese converting Islam realising muslim affected #Coronavirus #COVD country


[245, 9376, 4298, 856, 441, 19, 86]

In [88]:
test_label_index = Test['label'].apply(classes_list.index)
test_label1 = np.asarray(test_label_index)
test_label = to_categorical(np.asarray(test_label1))
test_label[0]

array([0., 1.], dtype=float32)

In [89]:
padded = sequence.pad_sequences(encoded1, maxlen=max_length, padding='post')
padded[0]

array([ 245, 9376, 4298,  856,  441,   19,   86,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [90]:
# testing

test_predictions = model.predict(padded)
# Finding out of two output neurons which one has highest probability
# It will return us the predicted class
# It will convert probability into final class 1 and 0
test_predictions1 = np.zeros_like(test_predictions)
test_predictions1[np.arange(len(test_predictions)), test_predictions.argmax(1)] = 1
#print(test_predictions)
#print(test_predictions1)

In [91]:
#Accuracy 
from sklearn.metrics import classification_report
print(classification_report(test_label,test_predictions1))

              precision    recall  f1-score   support

           0       0.92      0.93      0.92      1120
           1       0.92      0.91      0.92      1020

   micro avg       0.92      0.92      0.92      2140
   macro avg       0.92      0.92      0.92      2140
weighted avg       0.92      0.92      0.92      2140
 samples avg       0.92      0.92      0.92      2140

